In [ ]:
pip install pydub

In [ ]:
import numpy as np
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import IPython
#============Below 2 lines are written beacuse this assignment has been done in google colab=============
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection")

from td_utils import *
%matplotlib inline

In [ ]:
# Load preprocessed training examples
X = np.load("/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/XY_train/X.npy")
Y = np.load("/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/XY_train/Y.npy")

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

In [ ]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from tensorflow.keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
print(tensorflow.__version__)

1.15.0


In [ ]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/models/tr_model.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5511, 101)]       0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1375, 256)         388096    
_________________________________________________________________
batch_normalization_7 (Batch (None, 1375, 256)         1024      
_________________________________________________________________
activation_3 (Activation)    (None, 1375, 256)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1375, 256)         0         
_________________________________________________________________
gru_5 (GRU)                  (None, 1375, 128)         147840    
_________________________________________________________________
dropout_8 (Dropout)          (None, 1375, 128)         0   

In [ ]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
model.fit(X, Y, batch_size = 5, epochs=1)

Train on 26 samples
26/26 [==============================] - 22s 864ms/sample - loss: 0.0692 - acc: 0.9770


In [ ]:
# Load preprocessed dev set examples
X_dev = np.load("/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/XY_dev/X_dev.npy")
Y_dev = np.load("/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/XY_dev/Y_dev.npy")

In [ ]:
loss, acc = model.evaluate(X_dev, Y_dev)
print("Dev set accuracy = ", acc)

25/25 [==============================] - 1s 35ms/sample - loss: 0.3735 - acc: 0.9428
Dev set accuracy =  0.94280726


In [ ]:
def detect_triggerword(filename):
    plt.subplot(2, 1, 1)

    x = graph_spectrogram(filename)
    # the spectrogram outputs (freqs, Tx) and we want (Tx, freqs) to input into the model
    x  = x.swapaxes(0,1)
    x = np.expand_dims(x, axis=0)
    predictions = model.predict(x)
    
    plt.subplot(2, 1, 2)
    plt.plot(predictions[0,:,0])
    plt.ylabel('probability')
    plt.show()
    return predictions

In [ ]:
chime_file = "/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/audio_examples/chime.wav"
def chime_on_activate(filename, predictions, threshold):
    audio_clip = AudioSegment.from_wav(filename)
    chime = AudioSegment.from_wav(chime_file)
    Ty = predictions.shape[1]
    print(f"prediction shape is {prediction.shape}")
    # Step 1: Initialize the number of consecutive output steps to 0
    consecutive_timesteps = 0
    # Step 2: Loop over the output steps in the y
    for i in range(Ty):
        # Step 3: Increment consecutive output steps
        consecutive_timesteps += 1
        # Step 4: If prediction is higher than the threshold and more than 75 consecutive output steps have passed
        if predictions[0,i,0] > threshold and consecutive_timesteps > 75:
            # Step 5: Superpose audio and background using pydub
            audio_clip = audio_clip.overlay(chime, position = ((i / Ty) * audio_clip.duration_seconds)*1000)
            # Step 6: Reset consecutive output steps to 0
            consecutive_timesteps = 0
        
    audio_clip.export("chime_output.wav", format='wav')

In [ ]:
filename = "/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/raw_data/dev/1.wav"
prediction = detect_triggerword(filename)
chime_on_activate(filename, prediction, 0.5)
IPython.display.Audio("/content/drive/MyDrive/Colab Notebooks/Week 3(course5)/Trigger word detection/chime_output.wav")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(prediction)

[[[6.3691705e-02]
  [2.8078496e-02]
  [7.3859394e-03]
  ...
  [6.0802198e-05]
  [6.0802256e-05]
  [6.0802256e-05]]]
